# Chord charts and the Flavor Generator

In [2]:
import pandas as pd
import numpy as np
from chord import Chord
import random

In [3]:
# Load and set up data
dishes = pd.read_csv('Data/dish_components.csv',error_bad_lines=False)
numdish = len(dishes)
dishes = dishes.T
dishes = dishes.dropna(how='all')
compounds = pd.read_csv('Data/shared_compounds.csv')
combinatrix = compounds.groupby(['First Ingredient','Second Ingredient']).sum()

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (30,31,32) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Random dish to flavor Chord chart

In [46]:
# Pick out a random dish
rand_dish = int(np.random.randint(numdish, size=1))

In [47]:
# Look up the recipe, get the ingredients
recipe = dishes[rand_dish]
recipe = recipe.dropna()
recipe = recipe[1:][0:]
compon = [ing for ing in recipe]
recipe

Ingredient 1            butter
Ingredient 2             wheat
Ingredient 3          mushroom
Ingredient 4             yeast
Ingredient 5         asparagus
Ingredient 6    cheddar_cheese
Ingredient 7               egg
Name: 44934, dtype: object

In [48]:
# Loop through the ingredients and record values for shared flavor compounds
components = []

for i in range(0,len(compon)):
    for j in range(0,len(compon)):
        try: 
            components.append(int(combinatrix.loc[compon[i], compon[j]]))
        except KeyError:
            components.append(0)
    

In [49]:
# Make list into array
component_array = np.array(components).reshape((len(recipe),len(recipe)))

# Creates DataFrame and Matrix as a list.
cereals = pd.DataFrame(component_array, index=compon, columns=compon)

# Make symmetrical for chord chart
cereals = cereals + cereals.T

hops = cereals.values.tolist()

In [51]:
cereals

,butter,wheat,mushroom,yeast,asparagus,cheddar_cheese,egg
butter,0,1,51,0,0,71,28
wheat,0,0,0,0,0,0,0
mushroom,0,1,0,0,0,0,0
yeast,0,0,0,0,0,0,0
asparagus,30,3,25,0,0,28,14
cheddar_cheese,0,4,57,0,0,0,32
egg,0,0,24,0,0,0,0


In [43]:
compon

['butter', 'chicken_broth', 'parmesan_cheese', 'cauliflower', 'bread']

In [ ]:
# Random recipe Chord chart
# Chord(hops, compon).to_html('Dish' + str(rand_dish)+ '.html')

# Novel flavor combination generation

In [4]:
# Load in matrices from EDA.
shares = pd.read_csv("Shared_Compounds_Dense.csv")
recs = pd.read_csv("Recipe_Components_Dense.csv")

In [5]:
# Flavor generator function gives a list of flavors from a starting flavor
def flavor_generator(first_flavor, num_flavors, flavor_space):
    flavors = []
    flavors.append(first_flavor)
    n = num_flavors - 1
    current_flavor = first_flavor
    for i in range(0, n):
        new_flavor_index = random.choices(flavor_space.index,
                                          weights=flavor_space[current_flavor], k=1)
        new_flavor = flavor_space.iloc[new_flavor_index[0]][0]
        flavors.append(new_flavor)
        current_flavor = new_flavor
    return flavors

In [6]:
# Flavor dataframe function shows the flavor connections between ingredients in a list
def flavor_df(flavors):
    components = []

    for i in range(0,len(flavors)):
        for j in range(0,len(flavors)):
            try: 
                components.append(int(combinatrix.loc[flavors[i], flavors[j]]))
            except KeyError:
                components.append(0)
                
    flav_ar = np.array(components).reshape((len(flavors),len(flavors)))
    flav_df = pd.DataFrame(flav_ar, index=flavors, columns=flavors)
    flav_df = flav_df + flav_df.T
    
    return flav_df

In [7]:
# This is a result when applying the flavor generator to the recipes matrix
flavor_generator('coriander',7,recs)

['coriander',
 'potato',
 'garlic',
 'bell_pepper',
 'cayenne',
 'lemongrass',
 'peanut_butter']

In [7]:
# This is a result when applying the flavor generator to the shared compounds matrix
flavor_generator('enokidake',7,shares)

['enokidake',
 'raw_turkey',
 'boronia_ledifolia_leaf',
 'asparagus',
 'beer',
 'wort',
 'vanilla']

In [14]:
# Generate a few lists to use with the flavor dataframe function
flavor_6 = flavor_generator('cayenne',6,shares)
flavor_7 = flavor_generator('coriander',7,shares)
flavor_8 = flavor_generator('matsutake',7,shares)

In [10]:
fen = ['fennel','enokidake','coriander','marjoram','celery']
fenn = ['fennel', 'enokidake', 'feta_cheese', 'cherimoya', 'jamaican_rum']

In [12]:
# This is the cayenne dataframe
flavor_df(fenn)

,fennel,enokidake,feta_cheese,cherimoya,jamaican_rum
fennel,0,3,1,1,6
enokidake,3,0,52,17,56
feta_cheese,1,52,0,14,79
cherimoya,1,17,14,0,17
jamaican_rum,6,56,79,17,0


In [16]:
# This is the coriander dataframe
flavor_df(flavor_7)

,coriander,shrimp,citrus_peel_oil,soybean,dried_black_tea,strawberry,wild_strawberry
coriander,0,6,17,20,23,20,14
shrimp,6,0,2,47,39,27,11
citrus_peel_oil,17,2,0,20,29,29,19
soybean,20,47,20,0,82,70,31
dried_black_tea,23,39,29,82,0,86,36
strawberry,20,27,29,70,86,0,55
wild_strawberry,14,11,19,31,36,55,0


In [17]:
# This is the matsutake dataframe
flavor_df(flavor_8)

,matsutake,grape,clam,grilled_pork,roman_chamomile_oil,anise_hyssop,uncured_boiled_pork
matsutake,0,47,35,48,5,5,48
grape,47,0,22,33,10,2,33
clam,35,22,0,39,5,3,37
grilled_pork,48,33,39,0,7,3,115
roman_chamomile_oil,5,10,5,7,0,3,7
anise_hyssop,5,2,3,3,3,0,3
uncured_boiled_pork,48,33,37,115,7,3,0


In [19]:
# We can just take the values for our chord chart
lister = flavor_df(flavor_6).values.tolist()
lister

[[0, 8, 4, 1, 16, 8],
 [8, 0, 24, 18, 35, 27],
 [4, 24, 0, 36, 60, 92],
 [1, 18, 36, 0, 38, 41],
 [16, 35, 60, 38, 0, 88],
 [8, 27, 92, 41, 88, 0]]

In [20]:
# The labels for the chord chart is the input for the flavor dataframe function
#Chord(lister, flavor_7).to_html('cayenne_6.html')